### Two-star ELS

In [18]:
import numpy as np
import time

class Market: #market prarmeter를 관리하기 위한 class
    def __init__(self, rfr, correlation):
        self._rfr = rfr
        self._correlation = correlation

class Underlying:
    def __init__(self,refprice, spotvalue, volatility, dividend):
        self._refprice = refprice #기초자산 기준가
        self._spotvalue = spotvalue #기초자산 현재가
        self._volatility = volatility #변동성
        self._dividend = dividend #연속 배당률

def two_dimension_mc(obj_underlying_1: Underlying, obj_underlying_2: Underlying, obj_market: Market, 
                     redemption_schedule, coupon, full_dummy, barrier, ki_barrier, n_iteration):
    
    # 입력파라미터
    # obj_underlying_1 : 기초자산_1 class
    # obj_underlying_2 : 기초자산_2 class
    # objMarket : 시장 파라미터 class
    # redemption_schedule : 상환스케줄 array (발행시점부터 경과된 일자, 예를들어 
    # 6m = 125, 1y=250,..3y=750  -> (125, 250, 375, 500, 625, 750))
    # coupon : 상환시점에 수익상환시 지급되는 쿠폰 array (예컨대 {5%, 10%, 15%, 20%, 25%, 30%)
    # full_dummy : 만기시 full dummy 쿠폰 값
    # barrier : 상환시점의 배리어 array ( 예컨대, (0.9, 0.9, 0.85, 0.85, 0.8, 0.8))
    # ki_barrier : 낙인 배리어
    # n_iteration : MonteCarlo Simulation 횟수
    
    vol_1 = obj_underlying_1._volatility
    div_1 = obj_underlying_1._dividend
    current_spot_1 = obj_underlying_1._spotvalue
    refprice_1 = obj_underlying_1._refprice
    
    vol_2 = obj_underlying_2._volatility 
    div_2 = obj_underlying_2._dividend
    current_spot_2 = obj_underlying_2._spotvalue
    refprice_2 = obj_underlying_2._refprice

    rfr = obj_market._rfr
    corr = obj_market._correlation

    dt = 1 / 250
    maturity = redemption_schedule[-1]

    drift_1 = (rfr - div_1 -0.5*vol_1**2) * dt
    diffusion_1 = vol_1 * np.sqrt(dt)
    drift_2 = (rfr - div_2 -0.5*vol_2**2) * dt
    diffusion_2 = vol_2 * np.sqrt(dt)

    np.random.seed(0) # random number 시드고정
    sum_price = 0 # MC시뮬레이션 하나당 발생하는 결과를 누적할 변수
    redemp_prob = np.zeros(len(redemption_schedule) + 2) # 각 상환시점 쿠폰 상환 확률, full_dummy 지급확률, 손실상환 확률을 구할 변수
    start_time = time.time() #계산소요시간을 산출하기 위해 start time 기록

    for i in range(n_iteration):
        temp_price = 0
        payoff = 0
        discount_factor = 0
        s_series_1 = [current_spot_1]
        s_series_2 = [current_spot_2]
        old_spot_1 = current_spot_1
        old_spot_2 = current_spot_2

        for j in range(maturity):
            Z1 = np.random.standard_normal()
            Z2 = corr * Z1 + (1-corr**2) * np.random.standard_normal()

            new_spot_1 = old_spot_1 * np.exp(drift_1 + diffusion_1 * Z1)
            s_series_1.append(new_spot_1)
            old_spot_1 = new_spot_1

            new_spot_2 = old_spot_2 * np.exp(drift_2 + diffusion_2 * Z2)
            s_series_2.append(new_spot_2)
            old_spot_2 = new_spot_2

        s_series_1 = np.array(s_series_1)/refprice_1
        s_series_2 = np.array(s_series_2)/refprice_2

        for k in range(len(redemption_schedule)):
            idx = redemption_schedule[k]
            redemption_mat = idx
            if (s_series_1[idx] >= barrier[k]) and (s_series_2[idx] >= barrier[k]):
                payoff = 1 + coupon[k]
                discount_factor = np.exp(-rfr * redemption_mat/250)
                redemp_prob[k] += 1
                break
            else:
                discount_factor = np.exp(-rfr * redemption_mat/250)
                if (k==len(redemption_schedule)-1) & ((min(s_series_1)<ki_barrier) or (min(s_series_2)<ki_barrier)):
                    payoff = min(s_series_1[redemption_mat], s_series_2[redemption_mat])
                    redemp_prob[-1] += 1
                elif (k==len(redemption_schedule)-1) & ((min(s_series_1)>=ki_barrier) or (min(s_series_2)>=ki_barrier)):
                    payoff = 1 + full_dummy
                    redemp_prob[-2] += 1

        temp_price = payoff * discount_factor
        sum_price += temp_price

    els_value = sum_price/n_iteration

    cal_time = round(time.time() - start_time, 3)
    redemp_prob /= n_iteration
    return els_value, cal_time, redemp_prob


In [19]:
# 기초자산 1
refprice_1 = 100
spotvalue_1 = 100
volatility_1 = 0.2
dividend_1 = 0.001

#기초자산 2
refprice_2 = 100
spotvalue_2 = 100
volatility_2 = 0.4
dividend_2 = 0.02

underlying_1 = Underlying(refprice=refprice_1, spotvalue=spotvalue_1, volatility=volatility_1, dividend=dividend_1)
underlying_2 = Underlying(refprice=refprice_2, spotvalue=spotvalue_2, volatility=volatility_2, dividend=dividend_2)

correlation = 0.5
market = Market(rfr = 0.03, correlation=0.5)

redemption_schedule = np.array([1,2,3,4,5,6]) * 125 # 3y, 6m, 6 Chances stepdown ELS
coupon = np.array([1,2,3,4,5,6]) * 0.05
full_dummy = coupon[-1]
barrier = np.array([0.9, 0.9, 0.85, 0.8, 0.75, 0.70])
ki_barrier = 0.5
n_iteration = 1000

In [20]:
els_price, cal_time, prob = two_dimension_mc(underlying_1, underlying_2, market, redemption_schedule, coupon, 
                                             full_dummy, barrier, ki_barrier, n_iteration)

np.set_printoptions(suppress=True)
print(els_price)
print(cal_time)
print(prob)

0.9487413193435291
1.129
[0.559 0.089 0.067 0.044 0.039 0.027 0.01  0.165]
